In [1]:
import string
from collections import defaultdict

In [2]:
from bet import Bet

In [3]:
import sqlite3

conn = sqlite3.connect("bets_database.db")
cursor = conn.cursor()

#If the table does not exists, we create it
if cursor.execute("PRAGMA table_info(TABLE_TENNIS_PLAYERS)").fetchall() == []:
    tennis_players_table_query = """CREATE TABLE TABLE_TENNIS_PLAYERS (
        name TEXT UNIQUE
    );"""
    cursor.execute(tennis_players_table_query)
    conn.commit()

In [4]:
def retorno(ca, cb):
    return ca / (1 + (ca / cb))

In [15]:
from houses.House import *

class WilliamHill(House):
    def __init__(self):
        super().__init__()
        self.link = "https://sports.williamhill.es/betting/es-es"
        self.sports = {"tennis" : "tenis/partidos", "table_tennis" : "tenis-de-mesa/partidos/fecha/hoy"}
        self.ret_bets = {}
        self.separators = {"tennis" : "?", "table_tennis" : "₋"}
    
    def house(self):
        return "WilliamHill"

    def link_sport(self, sport):
        return f"{self.link}/{self.sports[sport]}"

    def sport_bets(self, sport):
        self.ret_bets[sport] = []
        self.driver.get(self.link_sport(sport))
        for _ in range(3):
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.2)
        response = self.driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        matches = soup.findAll('div', attrs={'class':'event'})
        for match in matches:
            teams = match.find('a')
            teams_final, bets_final = [], []
            teams_final = teams['title'].split(self.separators[sport])
            bets = match.findAll('button', attrs={'class':'btn betbutton oddsbutton'})
            for bet in bets:
                bets_final.append(bet.text)
            b = Bet(teams_final, bets_final)
            self.ret_bets[sport].append(b)
        self.real_names(sport)
        print("WilliamHill done")
        return self.ret_bets[sport]

    def real_names(self, sport):
        for bet in self.ret_bets[sport]:
            for team in bet.teams:
                cnt, l_f = 0, []
                for player in all_players[sport]:
                    if player[0] == team[0] or player[1] == team[1]:
                        cnt += 1
                        l_f.append((player, lev.jaro_winkler(player, team)))
                l_f.sort(key=lambda x: x[1], reverse=True)
                bet.real_teams.append(l_f[0][0])

In [16]:
%%time
w = WilliamHill()
w.sport_bets("table_tennis")
w.end()

table_tennis teamsfinal: ['Tynchtyk Turabekov ', ' Sergey Ogay']
table_tennis teamsfinal: ['Dmitriy Balakin ', ' Sergey Myagkov']
table_tennis teamsfinal: ['Sergey Martyukhin ', ' Pavel Semeshin']
table_tennis teamsfinal: ['Aleksandr Fedosov ', ' Roman Pavlenko']
table_tennis teamsfinal: ['Aleksey Vlasov ', ' Vasily Shirshov']
table_tennis teamsfinal: ['Arutyun Aslanyan ', ' Sergey Ogay']
table_tennis teamsfinal: ['Denis Sayanov ', ' Pavel Semeshin']
table_tennis teamsfinal: ['Oleg Kutuzov ', ' Sergey Myagkov']
table_tennis teamsfinal: ['Roman Pavlenko ', ' Tynchtyk Turabekov']
table_tennis teamsfinal: ['Vasily Shirshov ', ' Dmitriy Balakin']
table_tennis teamsfinal: ['Vladimir Safonov ', ' Sergey Martyukhin']
table_tennis teamsfinal: ['Sergey Ogay ', ' Aleksandr Fedosov']
table_tennis teamsfinal: ['Pavel Semeshin ', ' Yuriy Merkushin']
table_tennis teamsfinal: ['Sergey Myagkov ', ' Aleksey Vlasov']
table_tennis teamsfinal: ['Roman Pavlenko ', ' Arutyun Aslanyan']
table_tennis teamsfin

In [17]:
w.ret_bets

{'table_tennis': [TYNCHTYK TURABEKOV (2.62) x SERGEY OGAY (1.44),
  DMITRIY TUNITSYN (17.0) x SERGEY MYAGKOV (1.01),
  SERGEY MARTYUKHIN (15.0) x PAVEL SEMESHIN (1.02),
  ALEKSANDR KLAVDENKOV (1.73) x ROMAN PAVLENKO (2.0),
  ALEKSEY LOBANOV (2.0) x VASILY SHIRSHOV (1.73),
  ARUTYUN ASLANYAN (1.4) x SERGEY OGAY (2.75),
  DENIS SAYANOV (2.5) x PAVEL SEMESHIN (1.5),
  OLEG KUTUZOV (1.83) x SERGEY MYAGKOV (1.83),
  ROMAN PAVLENKO (1.44) x TYNCHTYK TURABEKOV (2.62),
  VASILY SHIRSHOV (1.83) x DMITRIY TUNITSYN (1.83),
  VLADIMIR SAFONOV (2.75) x SERGEY MARTYUKHIN (1.4),
  SERGEY OGAY (2.3) x ALEKSANDR KLAVDENKOV (1.57),
  PAVEL SEMESHIN (1.73) x YURII MERKUSHIN (2.0),
  SERGEY MYAGKOV (1.83) x ALEKSEY LOBANOV (1.83),
  ROMAN PAVLENKO (1.4) x ARUTYUN ASLANYAN (2.75),
  VASILY SHIRSHOV (2.0) x OLEG KUTUZOV (1.73),
  VLADIMIR SAFONOV (1.8) x DENIS SAYANOV (1.91),
  ALEKSANDR KLAVDENKOV (1.73) x TYNCHTYK TURABEKOV (2.0),
  ALEKSEY LOBANOV (1.73) x DMITRIY TUNITSYN (2.0),
  YURII MERKUSHIN (2.5) 

In [12]:
from houses.House import *

class Bwin(House):
    def __init__(self):
        super().__init__()
        self.link = "https://sports.bwin.es/es/sports"
        self.sports = {"tennis" : "tenis-5", "table_tennis" : "tenis-de-mesa-56/apuestas"}
        self.ret_bets = {}
        self.wait_element = {"tennis" : "#main-view > ms-widget-layout > ms-widget-slot.col-9.widget-slot", 
                             "table_tennis" : "#main-view > ms-widget-layout > ms-widget-slot.col-9.widget-slot"}

    def house(self):
        return "Bwin"

    def link_sport(self, sport):
        return f"{self.link}/{self.sports[sport]}"

    def sport_bets(self, sport):
        self.ret_bets[sport] = []
        self.driver.get(self.link_sport(sport))
        while True:
            try:
                self.driver.find_element_by_css_selector(self.wait_element[sport])
                break
            except:
                time.sleep(1)
                break
        response = self.driver.page_source
        soup = BeautifulSoup(response, "html.parser")        
        rows = soup.findAll('ms-event', attrs={'class': 'grid-event ms-active-highlight'})

        for row in rows:
            try:
                row = row.find('div', attrs={'class':'grid-event-wrapper'})
                teams = row.findAll('div', {'class':['participant-wrapper ng-star-inserted', 'participant-wrapper']})
                teams_final = []
                for team in teams:
                    team = team.find('div', attrs={'class':'participant'})
                    team = team.find(text=True, recursive=False).strip()
                    teams_final.append(team)

                bets = row.findAll('ms-option', attrs={'class':'grid-option'})
                bets_final = []
                for bet, _ in zip(bets, range(2)):
                    bets_final.append(bet.text)

                bet = Bet(teams_final, bets_final)
                self.ret_bets[sport].append(bet)
            except:
                pass
        self.real_names(sport)
        print("Bwin done")
        return self.ret_bets[sport]

    def real_names(self, sport):
        for bet in self.ret_bets[sport]:
            for team in bet.teams:
                cnt, l_f = 0, []
                for player in all_players[sport]:
                    if player[0] == team[0] or player[1] == team[1]:
                        cnt += 1
                        l_f.append((player, lev.jaro_winkler(player, team)))
                l_f.sort(key=lambda x: x[1], reverse=True)
                bet.real_teams.append(l_f[0][0])

In [13]:
%%time
b = Bwin()
b.sport_bets("table_tennis")
b.end()

Bwin done
Wall time: 11 s


In [14]:
b.ret_bets

{'table_tennis': [TYNCHTYK TURABEKOV (1.83) x SERGEY OGAY (1.87),
  SERGEY MARTYUKHIN (10.5) x PAVEL SEMESHIN (1.02),
  DMITRY BALAKIN (1.28) x SERGEY MYAGKOV (3.4),
  ALEKSANDR KLAVDENKOV (2.0) x ROMAN PAVLENKO (1.72),
  YURII MERKUSHIN (2.0) x VLADIMIR SAFONOV (1.72),
  ALEKSEY LOBANOV (2.0) x VASILY SHIRSHOV (1.72),
  ARUTYUN ASLANYAN (1.36) x SERGEY OGAY (2.9),
  DENIS SAYANOV (2.35) x PAVEL SEMESHIN (1.53),
  OLEG KUTUZOV (1.85) x SERGEY MYAGKOV (1.85),
  ROMAN PAVLENKO (1.42) x TYNCHTYK TURABEKOV (2.65),
  VLADIMIR SAFONOV (2.75) x SERGEY MARTYUKHIN (1.4),
  VASILY SHIRSHOV (1.85) x DMITRY BALAKIN (1.85),
  SERGEY OGAY (2.3) x ALEKSANDR KLAVDENKOV (1.55),
  PAVEL SEMESHIN (1.65) x YURII MERKUSHIN (2.1),
  SERGEY MYAGKOV (1.85) x ALEKSEY LOBANOV (1.85),
  ROMAN PAVLENKO (1.4) x ARUTYUN ASLANYAN (2.75),
  VLADIMIR SAFONOV (1.72) x DENIS SAYANOV (2.0),
  VASILY SHIRSHOV (2.0) x OLEG KUTUZOV (1.72),
  ALEKSANDR KLAVDENKOV (1.72) x TYNCHTYK TURABEKOV (2.0),
  YURII MERKUSHIN (2.45) x 